<a href="https://colab.research.google.com/github/mobarakol/PitVQA_Annotations/blob/main/Annotations_Creation/label_annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This part is to transfer video to images

In [ ]:
# global imports
import cv2
import numpy as np
import os

# strong typing
from pathlib import Path
from typing import List
from time import time

'''
Before run this code, your folder system should look like this:
project-name/
├── videos/
│   ├── video_01.mp4
│   └── video_02.mp4
│   └── ...
│   └── video_25.mp4
├── images/
├── video_to_images.py
note: 'images' is an empty folder before running this code.
'''


def main():
    """required variables are {pt_videos} and {pt_images}"""
    st = time()
    pt_videos = Path("videos")
    pt_images = Path("images")
    convert_videos_to_images(pt_videos=pt_videos, pt_images=pt_images)
    et = time()
    print(f'time used: {et-st}')


def convert_videos_to_images(pt_videos: Path, pt_images: Path):
    """convert all videos from {pt_videos} to images saved to {pt_images}"""
    create_directory(pt=pt_images)

    ls_videos: List[str] = os.listdir(pt_videos)
    ls_videos.sort()

    for str_video in ls_videos:
        pt_video: Path = pt_videos.joinpath(str_video)
        pt_image: Path = pt_images.joinpath(str_video.split(".")[0])

        create_directory(pt=pt_image)
        convert_video_to_image(pt_video=pt_video, pt_image=pt_image)
        print(f'{str_video} finished.')


def convert_video_to_image(pt_video: Path, pt_image: Path):
    """convert a single video from {pt_video} to images saved to {pt_image}"""
    video_capture = cv2.VideoCapture(str(pt_video))
    int_frames_per_second: int = np.ceil(video_capture.get(cv2.CAP_PROP_FPS))  # ceiling function to ensure integer

    int_frame: int = 0
    while video_capture.isOpened():
        bool_success, np_frame_matrix = video_capture.read()
        if bool_success:
            if int_frame % int_frames_per_second == 0:
                pt_image_frame: Path = pt_image.joinpath(f"{int(int_frame / int_frames_per_second):05}.png")
                cv2.imwrite(str(pt_image_frame), np_frame_matrix)
                print("written")
        else:
            break
        int_frame += 1

    video_capture.release()

    print(f"{pt_video} successfully converted to {int_frame} images.")


def create_directory(pt: Path):
    """create a directory for a given {path} if it does not already exist"""
    if not os.path.exists(pt):
        os.mkdir(pt)


main()


This part is to process the images

In [1]:
import os
import glob
from PIL import Image, ImageOps
from pathlib import Path

'''
Before run this code, your folder system should look like this:
project-name/
├── images/
│   ├── video_01/
│   └── video_02/
│   └── ...
│   └── video_25/
├── preprocessed_images/
├── preprocess_images.py
note: 'preprocessed_images' is an empty folder before running this code.
'''

if __name__ == '__main__':

    images_path = Path('./images')
    output_path = Path('./preprocessed_images')
    #video_sequence = [f"video_{i:02}" for i in range(1, 26)]'
    video_sequence = [f"video_{i:03}" for i in [18, 19, 33, 35]]


    for seq in video_sequence:
        folder_path = images_path / seq
        images = folder_path.glob('*.png')

        # create a new folder (e.g. video_01/) in the 'preprocessed_images' folder
        new_folder = output_path / seq
        new_folder.mkdir(parents=True, exist_ok=True)

        for img in images:
            image = Image.open(img)

            # process images
            crop_box = (295, 50, 935, 690)
            cropped_image = image.crop(crop_box)  # crop images
            resized_image = cropped_image.resize((224, 224), Image.BICUBIC)  # resize images

            # save processed images
            save_path = new_folder / img.name
            resized_image.save(save_path)
        print(seq + ' done.')


video_018 done.
video_019 done.
video_033 done.
video_035 done.
